# 🛡️ AWS CDK Guardrails Workflow - Complete Exercise Summary

**What we built**: A production-grade serverless guardrails pipeline using AWS CDK with Python

**Cost**: $0.00 (Free Tier)

---

## 📋 Table of Contents

1. [Architecture Overview](#architecture-overview)
2. [CDK Fundamentals](#cdk-fundamentals)
3. [Project Setup](#project-setup)
4. [Lambda Functions](#lambda-functions)
5. [The Stack Code](#the-stack-code)
6. [Deployment](#deployment)
7. [Testing](#testing)
8. [AWS Console Visualization](#aws-console-visualization)
9. [Cost Management](#cost-management)
10. [Cleanup](#cleanup)
11. [Quick Reference](#quick-reference)

---
## 🏗️ Architecture Overview <a name="architecture-overview"></a>

### The Workflow

```
User Query
    ↓
┌─────────────────┐
│ Lambda 1        │──→ INAPPROPRIATE ──→ [REJECT]
│ Input Guard     │
└────────┬────────┘
         ↓ OK
┌─────────────────┐
│ Lambda 2        │
│ GPT-4 (Mock)    │
└────────┬────────┘
         ↓
┌─────────────────┐
│ Lambda 3        │──→ INAPPROPRIATE ──→ [REJECT]
│ Output Guard    │
└────────┬────────┘
         ↓ OK
┌─────────────────┐
│ DynamoDB        │
│ Store Result    │
└─────────────────┘
         ↓
    GET Request → Read Result
```

### 💡 Sticky Analogy: Message Screening Facility

> **Think of it as a message screening facility:**
> - 📬 **Message arrives** (user query)
> - 🚨 **Security Guard #1** checks: "Is this message safe to process?"
> - 🤖 **Robot Writer** generates a response
> - 🚨 **Security Guard #2** checks: "Is this response appropriate?"
> - 📁 **Filing Cabinet** stores the approved result (DynamoDB)

### AWS Services Used

| Service | Purpose | Free Tier |
|---------|---------|----------|
| **Lambda** | Compute for guards and mock LLM | 1M requests/month |
| **Step Functions** | Orchestrates the workflow | 4,000 transitions/month |
| **DynamoDB** | Stores approved results | 25 GB storage |
| **S3** | CDK deployment artifacts | 5 GB |

---
## 🧱 CDK Fundamentals <a name="cdk-fundamentals"></a>

### What is AWS CDK?

**Infrastructure as Code** - Define your cloud resources using Python (or other languages) instead of clicking through the AWS Console.

> **Analogy**: CDK is like an **architect's blueprint**. You describe WHAT to build, and CDK figures out HOW to build it in AWS.

### Key CDK Modules

```python
from aws_cdk import (
    aws_stepfunctions as sfn,        # Workflow STRUCTURE
    aws_stepfunctions_tasks as tasks, # Actual WORK
    aws_lambda as lambda_,            # Compute functions
    aws_dynamodb as dynamodb,         # Database
)
```

### 💡 Sticky Analogy: Factory vs Machines

> **`aws_stepfunctions` (sfn)** = The **conveyor belt system**
> - Decides where things go
> - "If quality check passes, go left. If fails, go right."
>
> **`aws_stepfunctions_tasks` (tasks)** = The **machines on the belt**
> - Actually do the work
> - "This machine welds", "This machine paints"
>
> **One makes decisions. One does work.**

### CDK Module Comparison Table

| Module | What It Does | Example |
|--------|--------------|--------|
| `sfn.Choice` | Decision point | "If safe, continue. Else, reject." |
| `sfn.Fail` | Stop and report error | "Input Rejected" |
| `tasks.LambdaInvoke` | Call a Lambda function | "Run InputGuard" |
| `dynamodb.Table` | Create database table | Store results |

---
## 🚀 Project Setup <a name="project-setup"></a>

### Prerequisites Installed

```bash
# Node.js (required for CDK CLI)
brew install node

# CDK CLI
npm install -g aws-cdk

# AWS CLI
brew install awscli

# Configure credentials
aws configure set aws_access_key_id YOUR_KEY
aws configure set aws_secret_access_key YOUR_SECRET
aws configure set region us-east-1
```

### Project Initialization

```bash
mkdir -p ~/guardrails-demo
cd ~/guardrails-demo
cdk init app --language python
```

### Project Structure Created

```
~/guardrails-demo/
├── app.py                    # Entry point
├── cdk.json                  # CDK configuration
├── requirements.txt          # Python dependencies
├── .venv/                    # Virtual environment
├── lambda/                   # Our Lambda code (we create this)
│   ├── input_guard/
│   ├── gpt4_mock/
│   ├── output_guard/
│   └── store_result/
└── guardrails_demo/
    └── guardrails_demo_stack.py  # THE MAIN STACK CODE
```

---
## ⚡ Lambda Functions <a name="lambda-functions"></a>

### 1. Input Guard (`lambda/input_guard/index.py`)

**Purpose**: Block malicious inputs BEFORE they reach the LLM (saves money!)

In [ ]:
# lambda/input_guard/index.py
import json

def handler(event, context):
    """Mock input guardrail - checks for bad words"""
    user_input = event.get("query", "")
    
    # Simple mock: reject if contains "bad" or "hack"
    blocked_words = ["bad", "hack", "inject"]
    is_safe = not any(word in user_input.lower() for word in blocked_words)
    
    return {
        "query": user_input,
        "safe": is_safe,
        "stage": "input_guard"
    }

### 2. GPT-4 Mock (`lambda/gpt4_mock/index.py`)

**Purpose**: Simulate LLM response (in production, this calls GPT-4)

In [ ]:
# lambda/gpt4_mock/index.py
import json

def handler(event, context):
    """Mock LLM - returns a fake response"""
    user_query = event.get("query", "")
    
    # Fake LLM response (in production, you would call GPT-4 here)
    fake_response = "This is a helpful answer to: " + user_query
    
    return {
        "query": user_query,
        "response": fake_response,
        "stage": "gpt4_generation"
    }

### 3. Output Guard (`lambda/output_guard/index.py`)

**Purpose**: Check LLM output for toxic/harmful content

In [ ]:
# lambda/output_guard/index.py
import json

def handler(event, context):
    """Mock output guardrail - checks LLM response for bad content"""
    response = event.get("response", "")
    query = event.get("query", "")
    
    # Simple mock: reject if contains toxic words
    blocked_words = ["toxic", "harmful", "dangerous"]
    is_safe = not any(word in response.lower() for word in blocked_words)
    
    return {
        "query": query,
        "response": response,
        "safe": is_safe,
        "stage": "output_guard"
    }

### 4. Store Result (`lambda/store_result/index.py`)

**Purpose**: Save approved responses to DynamoDB

In [ ]:
# lambda/store_result/index.py
import json
import boto3
import os
import uuid

def handler(event, context):
    """Store the final result in DynamoDB"""
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table(os.environ["TABLE_NAME"])
    
    request_id = str(uuid.uuid4())
    
    table.put_item(Item={
        "request_id": request_id,
        "query": event.get("query", ""),
        "response": event.get("response", ""),
        "status": "completed"
    })
    
    return {
        "request_id": request_id,
        "status": "stored"
    }

---
## 📦 The Stack Code <a name="the-stack-code"></a>

This is the main infrastructure definition file.

`guardrails_demo/guardrails_demo_stack.py`

In [ ]:
from aws_cdk import (
    Duration,
    Stack,
    aws_stepfunctions as sfn,
    aws_stepfunctions_tasks as tasks,
    aws_lambda as lambda_,
    aws_dynamodb as dynamodb,
)
from constructs import Construct

class GuardrailsDemoStack(Stack):

    def __init__(self, scope: Construct, construct_id: str, **kwargs) -> None:
        super().__init__(scope, construct_id, **kwargs)

        # ========== DynamoDB Table ==========
        results_table = dynamodb.Table(self, "ResultsTable",
            partition_key=dynamodb.Attribute(
                name="request_id",
                type=dynamodb.AttributeType.STRING
            ),
            billing_mode=dynamodb.BillingMode.PAY_PER_REQUEST
        )

        # ========== Lambda Functions ==========
        input_guard = lambda_.Function(self, "InputGuard",
            runtime=lambda_.Runtime.PYTHON_3_11,
            handler="index.handler",
            code=lambda_.Code.from_asset("lambda/input_guard"),
            timeout=Duration.seconds(30)
        )

        gpt4_mock = lambda_.Function(self, "GPT4Mock",
            runtime=lambda_.Runtime.PYTHON_3_11,
            handler="index.handler",
            code=lambda_.Code.from_asset("lambda/gpt4_mock"),
            timeout=Duration.seconds(60)
        )

        output_guard = lambda_.Function(self, "OutputGuard",
            runtime=lambda_.Runtime.PYTHON_3_11,
            handler="index.handler",
            code=lambda_.Code.from_asset("lambda/output_guard"),
            timeout=Duration.seconds(30)
        )

        store_result = lambda_.Function(self, "StoreResult",
            runtime=lambda_.Runtime.PYTHON_3_11,
            handler="index.handler",
            code=lambda_.Code.from_asset("lambda/store_result"),
            timeout=Duration.seconds(10),
            environment={"TABLE_NAME": results_table.table_name}
        )
        results_table.grant_write_data(store_result)

        # ========== Step Functions Tasks ==========
        input_task = tasks.LambdaInvoke(self, "Check Input",
            lambda_function=input_guard,
            output_path="$.Payload"
        )

        gpt4_task = tasks.LambdaInvoke(self, "Generate Response",
            lambda_function=gpt4_mock,
            output_path="$.Payload"
        )

        output_task = tasks.LambdaInvoke(self, "Check Output",
            lambda_function=output_guard,
            output_path="$.Payload"
        )

        store_task = tasks.LambdaInvoke(self, "Store Result",
            lambda_function=store_result,
            output_path="$.Payload"
        )

        # ========== Failure States ==========
        reject_input = sfn.Fail(self, "Input Rejected",
            cause="Input failed safety check"
        )

        reject_output = sfn.Fail(self, "Output Rejected",
            cause="Output failed safety check"
        )

        # ========== Choice States ==========
        check_input_safe = sfn.Choice(self, "Input Safe?")
        check_output_safe = sfn.Choice(self, "Output Safe?")

        # ========== Wire Together ==========
        definition = (
            input_task
            .next(check_input_safe
                .when(sfn.Condition.boolean_equals("$.safe", False), reject_input)
                .otherwise(gpt4_task
                    .next(output_task)
                    .next(check_output_safe
                        .when(sfn.Condition.boolean_equals("$.safe", False), reject_output)
                        .otherwise(store_task)
                    )
                )
            )
        )

        sfn.StateMachine(self, "GuardrailsWorkflow",
            definition_body=sfn.DefinitionBody.from_chainable(definition),
            timeout=Duration.minutes(5)
        )

---
## 🚢 Deployment <a name="deployment"></a>

### Step 1: Bootstrap (One-Time Setup)

```bash
cd ~/guardrails-demo
source .venv/bin/activate
cdk bootstrap
```

> **Analogy**: Like setting up the construction site before building - creates storage shed (S3 bucket) and worker badges (IAM roles).

### Step 2: Deploy

```bash
cdk deploy --require-approval never
```

**What happens**:
1. CDK reads your stack code
2. Generates CloudFormation template
3. Creates all resources in AWS
4. Shows progress: `CREATE_IN_PROGRESS` → `CREATE_COMPLETE`
5. Final: `✅ GuardrailsDemoStack`

### Key CDK Commands Reference

| Command | Purpose |
|---------|--------|
| `cdk init app --language python` | Create new project |
| `cdk bootstrap` | One-time AWS setup |
| `cdk synth` | Preview CloudFormation template |
| `cdk deploy` | Deploy to AWS |
| `cdk diff` | Compare deployed vs current |
| `cdk destroy` | Delete all resources |

---
## 🧪 Testing <a name="testing"></a>

### Test 1: Happy Path ("hello" → Should Succeed)

```bash
# Create test input file (avoids shell escaping issues)
cat > ~/test-input.json << 'JSONEOF'
{"query":"hello"}
JSONEOF

# Start execution
aws stepfunctions start-execution \
  --state-machine-arn "arn:aws:states:us-east-1:ACCOUNT:stateMachine:NAME" \
  --input file://~/test-input.json

# Check result
aws stepfunctions describe-execution --execution-arn "arn:..."
```

**Result**: `"status": "SUCCEEDED"`

### Test 2: Rejection Path ("hack" → Should Fail)

```bash
cat > ~/test-bad.json << 'JSONEOF'
{"query":"hack"}
JSONEOF

aws stepfunctions start-execution \
  --state-machine-arn "arn:..." \
  --input file://~/test-bad.json
```

**Result**: 
```json
{
  "status": "FAILED",
  "cause": "Input failed safety check"
}
```

✅ **Guardrails worked!** The "hack" query was blocked before reaching GPT-4.

### Verify Data in DynamoDB

```bash
# List tables
aws dynamodb list-tables

# Scan table
aws dynamodb scan --table-name "GuardrailsDemoStack-ResultsTable-XXXXX"
```

**Result**:
```json
{
  "Items": [
    {
      "query": {"S": "hello"},
      "response": {"S": "This is a helpful answer to: hello"},
      "status": {"S": "completed"}
    }
  ]
}
```

---
## 🖥️ AWS Console Visualization <a name="aws-console-visualization"></a>

### How to Access

1. AWS Console → Search "Step Functions"
2. Click your state machine
3. Click any execution to see visual flow

### Visual Elements

| Element | Meaning |
|---------|--------|
| 🟢 Green | Succeeded step |
| 🔴 Red | Failed/rejected |
| 🔶 Diamond | Choice state (decision point) |
| 🟠 Orange | Task execution |

---
## 💰 Cost Management <a name="cost-management"></a>

### Check Costs Programmatically

```bash
# Check budget status
aws budgets describe-budgets --account-id YOUR_ACCOUNT_ID

# Check free tier usage
aws freetier get-free-tier-usage
```

### Our Usage Summary

| Service | Usage | Free Tier Limit | Cost |
|---------|-------|-----------------|------|
| Lambda | ~10 invocations | 1M/month | $0.00 |
| Step Functions | 3 executions | 4,000/month | $0.00 |
| DynamoDB | 2 writes, 1 scan | 25 GB | $0.00 |
| S3 | ~5 MB | 5 GB | $0.00 |
| **Total** | | | **$0.00** |

---
## 🗑️ Cleanup <a name="cleanup"></a>

### Delete All Resources

```bash
cd ~/guardrails-demo
source .venv/bin/activate
cdk destroy --force
```

### Why Some Resources Are Skipped

CDK defaults to `RETAIN` for stateful resources:

| Resource | Why Retained |
|----------|-------------|
| DynamoDB | Contains data (prevents accidental loss) |
| CloudWatch Logs | Contains audit trails |

> **Analogy**: Like a shredder that refuses to shred family photos, even if they're in the "to shred" pile. Better safe than sorry!

### To Force Delete (Dev/Test Only!)

```python
from aws_cdk import RemovalPolicy

table = dynamodb.Table(self, "Table",
    removal_policy=RemovalPolicy.DESTROY  # ⚠️ Deletes on cdk destroy
)
```

---
## 📚 Quick Reference <a name="quick-reference"></a>

### Complete Command Sequence

```bash
# 1. Setup
mkdir ~/guardrails-demo && cd ~/guardrails-demo
cdk init app --language python
source .venv/bin/activate
pip install -r requirements.txt

# 2. Create Lambda folders
mkdir -p lambda/{input_guard,gpt4_mock,output_guard,store_result}

# 3. Deploy
cdk bootstrap          # One-time
cdk deploy --require-approval never

# 4. Test
aws stepfunctions start-execution --state-machine-arn "..." --input file://test.json
aws stepfunctions describe-execution --execution-arn "..."

# 5. Cleanup
cdk destroy --force
```

### 💡 All Sticky Analogies

| Concept | Analogy |
|---------|--------|
| CDK | Architect's blueprint - describes WHAT, CDK builds HOW |
| `sfn` vs `tasks` | Factory: conveyor belt (decisions) vs machines (work) |
| Workflow | Message screening facility with security guards |
| `cdk bootstrap` | Setting up construction site before building |
| SageMaker Serverless | Magic kitchen that appears when needed |
| RemovalPolicy.RETAIN | Shredder that refuses to shred family photos |
| Idempotency | Elevator button - press 1x or 100x, same result |

---

## ✅ Exercise Complete!

**What you accomplished**:
- Built production-grade guardrails workflow
- Deployed with Infrastructure as Code
- Tested both success and failure paths
- Verified visually in AWS Console
- Managed costs ($0.00)
- Cleaned up properly


In [ ]:
#| hide

import subprocess
from pathlib import Path
from dialoghelper import curr_dialog

def deploy_notebook():
    nb_name = Path(curr_dialog()['name']).name + '.ipynb'
    src = f'/app/data/{curr_dialog()["name"]}.ipynb'
    dst = '/app/data/publish/portfolio/static/'
    print(nb_name)
    
    # Copy notebook to static folder
    subprocess.run(['cp', src, dst])
    
    # Deploy with plash
    subprocess.run(['plash_deploy'], cwd='/app/data/publish/portfolio')

In [ ]:
#| hide
deploy_notebook()